In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data

df=pd.read_csv('SpamBase_withoutdupl_norm_40.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att49,att50,att51,att52,att53,att54,att55,att56,att57,outlier
0,0.000000,0.044818,0.125490,0.0,0.032,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.023955,0.000000,0.000000,0.002502,0.006007,0.017487,1
1,0.046256,0.019608,0.098039,0.0,0.014,0.047619,0.028886,0.006301,0.000000,0.051705,...,0.000000,0.013536,0.0,0.011454,0.029985,0.002421,0.003735,0.010012,0.064836,1
2,0.013216,0.000000,0.139216,0.0,0.123,0.032313,0.026135,0.010801,0.121673,0.013751,...,0.002281,0.014664,0.0,0.008498,0.030651,0.000504,0.008008,0.048458,0.142551,1
3,0.000000,0.000000,0.000000,0.0,0.063,0.000000,0.042641,0.056706,0.058935,0.034653,...,0.000000,0.014048,0.0,0.004218,0.000000,0.000000,0.002303,0.003905,0.011995,1
4,0.000000,0.000000,0.000000,0.0,0.063,0.000000,0.042641,0.056706,0.058935,0.034653,...,0.000000,0.013843,0.0,0.004157,0.000000,0.000000,0.002303,0.003905,0.011995,1


In [3]:
df.shape

(4207, 58)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/SpamBase.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,57]
X = df[:,0:57]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
            # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 15:16:46,677][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 57)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 57)],y_test.shape=(1389,)
[ 2018-04-26 15:16:46,679][cascade_classifier.fit_transform] group_dims=[57]
[ 2018-04-26 15:16:46,681][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:16:46,682][cascade_classifier.fit_transform] group_ends=[57]
[ 2018-04-26 15:16:46,683][cascade_classifier.fit_transform] X_train.shape=(2818, 57),X_test.shape=(1389, 57)
[ 2018-04-26 15:16:46,685][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 57), X_cur_test.shape=(1389, 57)
[ 2018-04-26 15:16:47,498][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.41%
[ 2018-04-26 15:16:48,670][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=95.05%
[ 2018-04-26 15:16:49,615][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:16:57,672][cascade_classifier.transform] X_groups_test.shape=[(1389, 57)]
[ 2018-04-26 15:16:57,676][cascade_classifier.transform] group_dims=[57]
[ 2018-04-26 15:16:57,678][cascade_classifier.transform] X_test.shape=(1389, 57)
[ 2018-04-26 15:16:57,680][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 57)


Test Accuracy of ExtraTrees = 95.176386 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 15:17:00,109][cascade_classifier.transform] X_groups_test.shape=[(1389, 57)]
[ 2018-04-26 15:17:00,110][cascade_classifier.transform] group_dims=[57]
[ 2018-04-26 15:17:00,112][cascade_classifier.transform] X_test.shape=(1389, 57)
[ 2018-04-26 15:17:00,113][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 57)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 15:17:02,507][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 59)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 59)],y_test.shape=(1389,)
[ 2018-04-26 15:17:02,511][cascade_classifier.fit_transform] group_dims=[59]
[ 2018-04-26 15:17:02,512][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:17:02,513][cascade_classifier.fit_transform] group_ends=[59]
[ 2018-04-26 15:17:02,515][cascade_classifier.fit_transform] X_train.shape=(2818, 59),X_test.shape=(1389, 59)
[ 2018-04-26 15:17:02,520][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 59), X_cur_test.shape=(1389, 59)
[ 2018-04-26 15:17:03,372][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.05%
[ 2018-04-26 15:17:04,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.35%
[ 2018-04-26 15:17:05,411][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:17:13,047][cascade_classifier.transform] X_groups_test.shape=[(1389, 59)]
[ 2018-04-26 15:17:13,050][cascade_classifier.transform] group_dims=[59]
[ 2018-04-26 15:17:13,053][cascade_classifier.transform] X_test.shape=(1389, 59)
[ 2018-04-26 15:17:13,054][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 59)


Test Accuracy of ExtraTrees = 95.392369 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 15:17:15,565][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 59)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 59)],y_test.shape=(1389,)
[ 2018-04-26 15:17:15,567][cascade_classifier.fit_transform] group_dims=[59]
[ 2018-04-26 15:17:15,568][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:17:15,569][cascade_classifier.fit_transform] group_ends=[59]
[ 2018-04-26 15:17:15,570][cascade_classifier.fit_transform] X_train.shape=(2818, 59),X_test.shape=(1389, 59)
[ 2018-04-26 15:17:15,572][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 59), X_cur_test.shape=(1389, 59)
[ 2018-04-26 15:17:16,382][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.41%
[ 2018-04-26 15:17:17,622][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.35%
[ 2018-04-26 15:17:18,740][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 95.248380 %', 0, 'fois')
(' Time ', '11.459', ' seconds')


[ 2018-04-26 15:17:30,412][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.11%
[ 2018-04-26 15:17:31,533][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.35%
[ 2018-04-26 15:17:32,676][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.81%
[ 2018-04-26 15:17:33,842][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.04%
[ 2018-04-26 15:17:34,893][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=95.39%
[ 2018-04-26 15:17:35,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=95.74%
[ 2018-04-26 15:17:37,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=93.24%
[ 2018-04-26 15:17:38,111][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=93.59%


('Test Accuracy of ExtraTrees = 94.816415 %', 1, 'fois')
(' Time ', '11.109', ' seconds')


[ 2018-04-26 15:17:44,042][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.47%
[ 2018-04-26 15:17:45,133][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.35%
[ 2018-04-26 15:17:46,226][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.81%
[ 2018-04-26 15:17:47,256][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.39%
[ 2018-04-26 15:17:48,347][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=95.39%
[ 2018-04-26 15:17:49,511][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=95.74%
[ 2018-04-26 15:17:50,639][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=92.88%
[ 2018-04-26 15:17:51,768][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=93.24%


...

('Test Accuracy of ExtraTrees = 94.600432 %', 17, 'fois')
(' Time ', '10.658', ' seconds')


[ 2018-04-26 15:21:16,414][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.11%
[ 2018-04-26 15:21:17,488][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.99%
[ 2018-04-26 15:21:18,568][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=93.97%
[ 2018-04-26 15:21:19,550][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=96.10%
[ 2018-04-26 15:21:20,656][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=95.39%
[ 2018-04-26 15:21:21,603][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=94.68%
[ 2018-04-26 15:21:22,853][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=92.88%
[ 2018-04-26 15:21:23,934][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=93.95%


('Test Accuracy of ExtraTrees = 94.600432 %', 18, 'fois')
(' Time ', '10.743', ' seconds')


[ 2018-04-26 15:21:29,648][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.41%
[ 2018-04-26 15:21:30,684][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.29%
[ 2018-04-26 15:21:31,791][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=93.97%
[ 2018-04-26 15:21:32,849][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.74%
[ 2018-04-26 15:21:33,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=95.74%
[ 2018-04-26 15:21:34,935][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=94.68%
[ 2018-04-26 15:21:35,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=92.17%
[ 2018-04-26 15:21:37,076][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=94.31%


('Test Accuracy of ExtraTrees = 94.744420 %', 19, 'fois')
(' Time ', '10.387', ' seconds')


# ('Test Accuracy of ExtraTrees = 95.248380 %', 0, 'fois')

# (' Time ', '11.459', ' seconds')